## Шаг 1. Загрузка и очистка данных

### Скачиваем датасет [InstaCart Online Grocery Basket Analysis Dataset](https://www.kaggle.com/datasets/yasserh/instacart-online-grocery-basket-analysis-dataset/data)

In [1]:
pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install kaggle

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Настройка прав доступа 
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d "yasserh/instacart-online-grocery-basket-analysis-dataset"

Dataset URL: https://www.kaggle.com/datasets/yasserh/instacart-online-grocery-basket-analysis-dataset
License(s): CC0-1.0
  0%|                                                | 0.00/197M [00:00<?, ?B/s]
100%|████████████████████████████████████████| 197M/197M [00:00<00:00, 2.30GB/s]


In [7]:
!unzip instacart-online-grocery-basket-analysis-dataset.zip
!rm instacart-online-grocery-basket-analysis-dataset.zip

Archive:  instacart-online-grocery-basket-analysis-dataset.zip
  inflating: aisles.csv              
  inflating: departments.csv         
  inflating: order_products__prior.csv  
  inflating: order_products__train.csv  
  inflating: orders.csv              
  inflating: products.csv            


In [3]:
import os
import pandas as pd

# Фильтруем только CSV-файлы
csv_files = [f for f in os.listdir() if f.endswith('.csv')]

for file in csv_files:
    # Формируем имя переменной
    var_name = file[:-4]  # Удаляем .csv
    # Создаем датафрейм в глобальном пространстве имен
    globals()[var_name] = pd.read_csv(file)
    print(f'Файл {file} загружен в переменную {var_name}')

Файл order_products__prior.csv загружен в переменную order_products__prior
Файл products.csv загружен в переменную products
Файл orders.csv загружен в переменную orders
Файл order_products__train.csv загружен в переменную order_products__train
Файл aisles.csv загружен в переменную aisles
Файл departments.csv загружен в переменную departments


In [7]:
order_products__prior.head(5)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [8]:
products.head(5)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [9]:
orders.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [10]:
order_products__train.head(5)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [11]:
aisles.head(5)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [12]:
departments.head(5)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [13]:
# 1. Объединяем информацию о товарах в заказах
combined_orders_products = pd.concat(
    [order_products__prior, order_products__train], 
    ignore_index=True
)

# Проверка полных дубликатов строк
print("Дубликаты в объединенных заказах:",
      combined_orders_products.duplicated().sum())

# Проверка логических дубликатов (один товар в одном заказе)
print("Логические дубликаты (order_id + product_id):",
      combined_orders_products.duplicated(subset=['order_id', 'product_id']).sum())

# 2. Соединяем с метаданными заказов
merged_data = orders.merge(combined_orders_products, on='order_id', how='left')

# Проверка комбинации order_id + product_id
duplicates = merged_data.duplicated(subset=['order_id', 'product_id'], keep=False)
print("Потенциальные дубликаты после объединения с orders:",
      duplicates.sum())

# 3. Добавляем информацию о продуктах
merged_data = merged_data.merge(products, on='product_id', how='left')

# Проверка уникальности связки order + product
print("Дубликаты после добавления продуктов:",
      merged_data.duplicated(subset=['order_id', 'product_id']).sum())

# 4. Добавляем информацию об отделах (aisles)
merged_data = merged_data.merge(
    aisles,
    on='aisle_id',
    how='left'
)

# 5. Добавляем информацию о категориях (departments)
merged_data = merged_data.merge(
    departments,
    on='department_id',
    how='left'
)

final_duplicates = merged_data.duplicated(
    subset=['order_id', 'product_id', 'aisle_id', 'department_id']
).sum()

print("Итоговое количество дубликатов:", final_duplicates)

merged_data = merged_data.drop(['eval_set'], axis=1)

df = merged_data

Дубликаты в объединенных заказах: 0
Логические дубликаты (order_id + product_id): 0
Потенциальные дубликаты после объединения с orders: 0
Дубликаты после добавления продуктов: 0
Итоговое количество дубликатов: 0


In [14]:
df[df['user_id']==1]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,77.0,7.0,soft drinks,beverages
1,2539329,1,1,2,8,NaN,14084.0,2.0,0.0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,soy lactosefree,dairy eggs
2,2539329,1,1,2,8,NaN,12427.0,3.0,0.0,Original Beef Jerky,23.0,19.0,popcorn jerky,snacks
3,2539329,1,1,2,8,NaN,26088.0,4.0,0.0,Aged White Cheddar Popcorn,23.0,19.0,popcorn jerky,snacks
4,2539329,1,1,2,8,NaN,26405.0,5.0,0.0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,paper goods,household
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,1187899,1,11,4,8,14.0,13032.0,7.0,1.0,Cinnamon Toast Crunch,121.0,14.0,cereal,breakfast
66,1187899,1,11,4,8,14.0,26088.0,8.0,1.0,Aged White Cheddar Popcorn,23.0,19.0,popcorn jerky,snacks
67,1187899,1,11,4,8,14.0,27845.0,9.0,0.0,Organic Whole Milk,84.0,16.0,milk,dairy eggs
68,1187899,1,11,4,8,14.0,49235.0,10.0,1.0,Organic Half & Half,53.0,16.0,cream,dairy eggs


In [146]:
from tabulate import tabulate
# print(tabulate(df[df['user_id']==1], headers='keys', tablefmt='pipe', showindex=False))

print('order_products__prior \n')
print(tabulate(order_products__prior.head(), headers='keys', tablefmt='pipe', showindex=False))
print('products \n')
print(tabulate(products.head(), headers='keys', tablefmt='pipe', showindex=False))
print('orders \n')
print(tabulate(orders.head(), headers='keys', tablefmt='pipe', showindex=False))
print('order_products__train \n')
print(tabulate(order_products__train.head(), headers='keys', tablefmt='pipe', showindex=False))
print('aisles \n')
print(tabulate(aisles.head(), headers='keys', tablefmt='pipe', showindex=False))
print('departments \n')
print(tabulate(departments.head(), headers='keys', tablefmt='pipe', showindex=False))

order_products__prior 

|   order_id |   product_id |   add_to_cart_order |   reordered |
|-----------:|-------------:|--------------------:|------------:|
|          2 |        33120 |                   1 |           1 |
|          2 |        28985 |                   2 |           1 |
|          2 |         9327 |                   3 |           0 |
|          2 |        45918 |                   4 |           1 |
|          2 |        30035 |                   5 |           0 |
products 

|   product_id | product_name                                                      |   aisle_id |   department_id |
|-------------:|:------------------------------------------------------------------|-----------:|----------------:|
|            1 | Chocolate Sandwich Cookies                                        |         61 |              19 |
|            2 | All-Seasons Salt                                                  |        104 |              13 |
|            3 | Robust Golden Unsweete

## EDA

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter

# Инициализация стиля графиков
plt.style.use('seaborn-v0_8') 
sns.set_palette("husl")

In [22]:
# 1. Общая информация о данных
print("Первые 5 строк:")
display(df.head())
print("\nИнформация о данных:")
df.info()
print("\nОписательная статистика:")
display(df.describe(include='all'))

Первые 5 строк:


,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,77.0,7.0,soft drinks,beverages
1,2539329,1,1,2,8,NaN,14084.0,2.0,0.0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,soy lactosefree,dairy eggs
2,2539329,1,1,2,8,NaN,12427.0,3.0,0.0,Original Beef Jerky,23.0,19.0,popcorn jerky,snacks
3,2539329,1,1,2,8,NaN,26088.0,4.0,0.0,Aged White Cheddar Popcorn,23.0,19.0,popcorn jerky,snacks
4,2539329,1,1,2,8,NaN,26405.0,5.0,0.0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,paper goods,household



Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33894106 entries, 0 to 33894105
Data columns (total 14 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_number            int64  
 3   order_dow               int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              float64
 7   add_to_cart_order       float64
 8   reordered               float64
 9   product_name            object 
 10  aisle_id                float64
 11  department_id           float64
 12  aisle                   object 
 13  department              object 
dtypes: float64(6), int64(5), object(3)
memory usage: 3.5+ GB

Описательная статистика:


,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
count,3.389411e+07,3.389411e+07,3.389411e+07,3.389411e+07,3.389411e+07,3.181604e+07,3.381911e+07,3.381911e+07,3.381911e+07,33819106,3.381911e+07,3.381911e+07,33819106,33819106
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49685,NaN,NaN,134,21
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Banana,NaN,NaN,fresh fruits,produce
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,491291,NaN,NaN,3792661,9888378
mean,1.710564e+06,1.029445e+05,1.713871e+01,2.737388e+00,1.343158e+01,1.137763e+01,2.557551e+04,8.367738e+00,5.900617e-01,NaN,7.121799e+01,9.918544e+00,NaN,NaN
std,9.874018e+05,5.946732e+04,1.749646e+01,2.093372e+00,4.246102e+00,8.949251e+00,1.409770e+04,7.139540e+00,4.918220e-01,NaN,3.819898e+01,6.281655e+00,NaN,NaN
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,NaN,1.000000e+00,1.000000e+00,NaN,NaN
25%,8.554130e+05,5.143500e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,1.351900e+04,3.000000e+00,0.000000e+00,NaN,3.100000e+01,4.000000e+00,NaN,NaN
50%,1.710659e+06,1.026270e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00,2.525600e+04,6.000000e+00,1.000000e+00,NaN,8.300000e+01,9.000000e+00,NaN,NaN
75%,2.565583e+06,1.544110e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.500000e+01,3.793500e+04,1.100000e+01,1.000000e+00,NaN,1.070000e+02,1.600000e+01,NaN,NaN


In [ ]:
# 2. Анализ пропущенных значений
plt.figure(figsize=(10, 4))
sns.heatmap(df.isnull(), cbar=False, yticklabels=False)
plt.title('Распределение пропущенных значений')
plt.show()

## BaseLine

### Popularity-Based Recommendations

Metrics:
- precision@10: 0.0290
- recall@10: 0.0304
- hit_rate@10: 0.2768
- coverage: 0.0003

In [18]:
df = merged_data

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import defaultdict

# 1. Подготовка данных --------------------------------------------------------
# Убираем дубликаты (один товар в одном заказе учитываем один раз)
clean_df = df.drop_duplicates(['user_id', 'product_id'])

# Разделение на train/test (80/20) по пользователям
train_users, test_users = train_test_split(
    clean_df['user_id'].unique(), 
    test_size=0.2, 
    random_state=42
)

train = clean_df[clean_df['user_id'].isin(train_users)]
test = clean_df[clean_df['user_id'].isin(test_users)]

In [20]:
# 2. Расчет популярности на тренировочных данных ------------------------------
def calculate_popularity(train_data, alpha=1.0):
    """Рассчитывает популярность с smoothing-регуляризацией"""
    item_popularity = train_data.groupby('product_id')['reordered'].agg(['count', 'sum'])
    item_popularity['score'] = (item_popularity['sum'] + alpha) / (item_popularity['count'] + alpha*2)
    return item_popularity.sort_values('score', ascending=False)

popularity_df = calculate_popularity(train, alpha=10)

In [21]:
# 3. Генерация рекомендаций ---------------------------------------------------
def generate_recommendations(popularity_df, k=10):
    """Генерирует топ-K популярных рекомендаций"""
    return popularity_df.head(k).index.tolist()

top_k_recommendations = generate_recommendations(popularity_df, k=10)

In [22]:
# 4. Оценка качества ----------------------------------------------------------
def evaluate_recommendations(test_data, recommendations):
    # Собираем купленные товары для каждого пользователя в тесте
    test_users_items = test_data.groupby('user_id')['product_id'].apply(set)
    
    # Считаем метрики
    hit = 0
    precision_sum = 0
    recall_sum = 0
    n_users = len(test_users_items)
    
    recommended_set = set(recommendations)
    
    for user_items in test_users_items:
        # Пересечение рекомендованных и реальных товаров
        intersection = user_items & recommended_set
        
        # Hit Rate
        hit += 1 if len(intersection) > 0 else 0
        
        # Precision@K
        precision = len(intersection) / len(recommendations)
        precision_sum += precision
        
        # Recall@K
        recall = len(intersection) / len(user_items) if len(user_items) > 0 else 0
        recall_sum += recall
    
    coverage = len(recommended_set) / len(popularity_df)
    
    return {
        'precision@10': precision_sum / n_users,
        'recall@10': recall_sum / n_users,
        'hit_rate@10': hit / n_users,
        'coverage': coverage
    }

# Вычисляем метрики
metrics = evaluate_recommendations(test, top_k_recommendations)

In [23]:
# 5. Вывод результатов --------------------------------------------------------
print("Top 10 Popular Recommendations:")
print(popularity_df[['score']].head(10))

print("\nMetrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

Top 10 Popular Recommendations:
              score
product_id         
49423.0     0.47619
49487.0     0.47619
49498.0     0.47619
49540.0     0.47619
49627.0     0.47619
29427.0     0.47619
29542.0     0.47619
49206.0     0.47619
49238.0     0.47619
3659.0      0.47619

Metrics:
precision@10: 0.0000
recall@10: 0.0000
hit_rate@10: 0.0003
coverage: 0.0002


### Косинусное Сходство
Metrics:
- precision@10: 0.0239
- recall@10: 0.0227
- hit_rate@10: 0.2280
- coverage: 0.0003

In [29]:
df = merged_data

In [30]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix, save_npz, load_npz
from tqdm import tqdm
import os

# 1. Подготовка данных --------------------------------------------------------
# Уникальные взаимодействия пользователь-товар
clean_df = df.drop_duplicates(['user_id', 'product_id'])

# Разделение на train/test (80/20) по пользователям
train_users, test_users = train_test_split(
    clean_df['user_id'].unique(), 
    test_size=0.2, 
    random_state=42
)

train = clean_df[clean_df['user_id'].isin(train_users)]
test = clean_df[clean_df['user_id'].isin(test_users)]

In [31]:
# 2. Создание матрицы взаимодействий (исправленная версия)
def create_item_user_matrix(data):
    """Создает разреженную матрицу товар-пользователь"""
    # Явное сохранение категориальных соответствий
    data = data.copy()
    data['user_code'] = data['user_id'].astype('category').cat.codes
    data['product_code'] = data['product_id'].astype('category').cat.codes
    
    # Создаем матрицу с явным указанием размеров
    n_products = data['product_code'].max() + 1
    n_users = data['user_code'].max() + 1
    
    matrix = csr_matrix(
        (np.ones(len(data)), 
        (data['product_code'], data['user_code'])),
        shape=(n_products, n_users),
        dtype='float32'
    )
    return matrix, data['product_id'].astype('category').cat.categories

# Создаем и сохраняем матрицу для ускорения повторных запусков
if not os.path.exists('item_user_matrix.npz'):
    item_user_matrix, product_ids = create_item_user_matrix(train)
    save_npz('item_user_matrix.npz', item_user_matrix)
    pd.Series(product_ids).to_csv('product_ids.csv', index=False)
else:
    item_user_matrix = load_npz('item_user_matrix.npz')
    product_ids = pd.read_csv('product_ids.csv').squeeze()

ValueError: negative axis 0 index: -1

In [60]:
# 3. Расчет косинусного сходства ----------------------------------------------
if not os.path.exists('cosine_sim.npy'):
    # Исправляем нормализацию
    l2_norm = np.sqrt(item_user_matrix.power(2).sum(axis=1)).A.ravel()
    l2_norm = np.where(l2_norm == 0, 1.0, l2_norm)  # Замена нулей на 1
    
    item_user_matrix_normalized = item_user_matrix.multiply(1 / l2_norm[:, None])
    cosine_sim = cosine_similarity(item_user_matrix_normalized, dense_output=False)
    np.save('cosine_sim.npy', cosine_sim)

In [61]:
# 4. Генерация рекомендаций ---------------------------------------------------
def recommend_item_item(user_id, k=10, history_items=5, n_similar=5):
    user_history = train[train['user_id'] == user_id]['product_id'].values
    if len(user_history) == 0:
        # Для холодных пользователей возвращаем популярное
        return popularity_df.head(k).index.tolist()
    
    recent_items = user_history[-history_items:]
    similar_items = []
    
    # Создаем словарь для быстрого поиска кодов
    product_code_map = {pid: idx for idx, pid in enumerate(product_ids)}
    
    for item in recent_items:
        if item in product_code_map:
            item_code = product_code_map[item]
            sim_scores = cosine_sim[item_code].toarray().ravel()
            top_indices = np.argsort(-sim_scores)[1:n_similar+1]
            similar_items.extend([product_ids[i] for i in top_indices])
    
    # Удаляем уже купленные
    purchased = set(user_history)
    recommendations = [x for x in np.unique(similar_items) if x not in purchased]
    
    # Если рекомендаций мало, дополняем популярными
    if len(recommendations) < k:
        top_popular = [pid for pid in popularity_df.index if pid not in purchased]
        recommendations += top_popular[:k - len(recommendations)]
    
    return recommendations[:k]

In [62]:
# 5. Оценка качества ----------------------------------------------------------
def evaluate_item_item(test_data, recommend_func):
    test_users = test_data.groupby('user_id').filter(lambda x: len(x) > 0)['user_id'].unique()
    metrics = {
        'precision@10': 0,
        'recall@10': 0,
        'hit_rate@10': 0,
        'coverage': set()
    }
    
    for user in tqdm(test_users[:1000]):  # Ограничиваем выборку для демонстрации
        # Получаем рекомендации
        recommendations = recommend_func(user)
        
        # Получаем реальные покупки из теста
        actual = test_data[test_data['user_id'] == user]['product_id'].values
        
        if len(recommendations) == 0 or len(actual) == 0:
            continue
        
        # Вычисляем метрики
        intersection = set(recommendations) & set(actual)
        
        # Precision
        metrics['precision@10'] += len(intersection) / len(recommendations)
        
        # Recall
        metrics['recall@10'] += len(intersection) / len(actual)
        
        # Hit Rate
        metrics['hit_rate@10'] += 1 if len(intersection) > 0 else 0
        
        # Coverage
        metrics['coverage'].update(recommendations)
    
    # Усредняем метрики
    n_users = len(test_users[:1000])
    metrics['precision@10'] /= n_users
    metrics['recall@10'] /= n_users
    metrics['hit_rate@10'] /= n_users
    metrics['coverage'] = len(metrics['coverage']) / len(product_ids)
    
    return metrics

# Запуск оценки
metrics = evaluate_item_item(test, recommend_item_item)

100%|█████████████████████████████████████| 1000/1000 [00:00<00:00, 1327.60it/s]


In [63]:
# 6. Вывод результатов --------------------------------------------------------
print("\nItem-Item Cosine Similarity Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")


Item-Item Cosine Similarity Metrics:
precision@10: 0.0239
recall@10: 0.0227
hit_rate@10: 0.2280
coverage: 0.0003


In [64]:
# После загрузки product_ids
print("Пример соответствия ID и кодов:")
print(pd.DataFrame({'product_id': product_ids[:10], 'code': range(10)}))

Пример соответствия ID и кодов:
   product_id  code
0           1     0
1           2     1
2           3     2
3           4     3
4           5     4
5           8     5
6           9     6
7          10     7
8          11     8
9          12     9


In [65]:
# Для случайного товара
sample_product = product_ids[100]
print(f"Сходство для {sample_product}:")
print(cosine_sim[100].toarray())

Сходство для 123:
[[0. 0. 0. ... 0. 0. 0.]]


In [66]:
test_user = test['user_id'].iloc[0]
print(f"\nТест для пользователя {test_user}:")
recs = recommend_item_item(test_user)
print("Рекомендации:", recs)
print("Реальные покупки:", test[test['user_id'] == test_user]['product_id'].values)


Тест для пользователя 1:
Рекомендации: [24852, 20940, 38689, 39180, 12384, 24024, 29447, 13176, 1729, 44799]
Реальные покупки: [  196 25133 38928 26405 39657 10258 13032 26088 27845 49235 46149]


### Матричная факторизация с использованием Alternating Least Squares (ALS)
Metrics:
- precision@10: 0.0008
- recall@10: 0.0002
- hit_rate@10: 0.0030
- coverage: 0.0219

In [68]:
pip install implicit

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 9.4 MB/s eta 0:00:0011.6 MB/s eta 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [97]:
import implicit
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split

# 1. Подготовка данных --------------------------------------------------------
# Используем уже очищенные данные из предыдущих шагов
train = clean_df[clean_df['user_id'].isin(train_users)]
test = clean_df[clean_df['user_id'].isin(test_users)]

# Создаем категории для ВСЕХ пользователей и товасов из полного датасета
all_users = df['user_id'].unique()
all_products = df['product_id'].unique()

user_cat = pd.CategoricalDtype(categories=all_users, ordered=True)
product_cat = pd.CategoricalDtype(categories=all_products, ordered=True)

In [98]:
# 2. Создание user-item матрицы (финальная версия) -----------------------------
def create_user_item_matrix(data):
    """Создает матрицу взаимодействий с глобальными категориями"""
    users = data['user_id'].astype(user_cat).cat.codes
    items = data['product_id'].astype(product_cat).cat.codes
    
    weights = 1 + 0.5*data['reordered'].values
    
    return csr_matrix(
        (weights, (users, items)),
        shape=(len(user_cat.categories), len(product_cat.categories)),  # Фиксированный размер
        dtype=np.float32
    )

In [99]:
# 3. Переинициализация матрицы и модели ----------------------------------------
user_item_matrix = create_user_item_matrix(train)
item_user_matrix = user_item_matrix.T.tocsc()

model = implicit.als.AlternatingLeastSquares(
    factors=64,
    iterations=15,
    regularization=0.1,
    random_state=42
)
model.fit(item_user_matrix)

100%|███████████████████████████████████████████| 15/15 [00:03<00:00,  3.77it/s]


In [100]:
# 4. Генерация рекомендаций --------------------------------------------------
def recommend_als(user_id, k=10):
    try:
        user_code = user_cat.categories.get_loc(user_id)
        items, _ = model.recommend(
            userid=user_code,
            user_items=user_item_matrix[user_code],
            N=k,
            filter_already_liked_items=True
        )
        
        # Проверка валидности индексов
        valid_items = [idx for idx in items if idx < len(product_cat.categories)]
        return [product_cat.categories[idx] for idx in valid_items][:k]
    
    except KeyError:
        # Гибридные рекомендации для новых пользователей
        top_popular = popularity_df[popularity_df.index.isin(product_cat.categories)]
        return top_popular.head(k).index.tolist()

In [101]:
# 5. Оценка качества ---------------------------------------------------------
def evaluate_als(test_data, recommend_func):
    test_users = test_data['user_id'].unique()
    metrics = {
        'precision@10': 0,
        'recall@10': 0,
        'hit_rate@10': 0,
        'coverage': set()  # Для сбора уникальных рекомендованных товаров
    }
    
    for user in tqdm(test_users[:1000]):
        recommendations = recommend_func(user)
        actual = test_data[test_data['user_id'] == user]['product_id'].values
        
        if len(recommendations) == 0 or len(actual) == 0:
            continue
            
        intersection = set(recommendations) & set(actual)
        
        # Расчет базовых метрик
        metrics['precision@10'] += len(intersection) / len(recommendations)
        metrics['recall@10'] += len(intersection) / len(actual)
        metrics['hit_rate@10'] += 1 if len(intersection) > 0 else 0
        
        # Обновление покрытия
        metrics['coverage'].update(recommendations)
    
    # Усреднение метрик
    n_users = len(test_users[:1000])
    metrics = {
        'precision@10': metrics['precision@10'] / n_users,
        'recall@10': metrics['recall@10'] / n_users,
        'hit_rate@10': metrics['hit_rate@10'] / n_users,
        'coverage': len(metrics['coverage']) / len(product_cat.categories)
    }
    
    return metrics

In [102]:
# 6. Вывод результатов -------------------------------------------------------
print("\nALS Metrics:")
als_metrics = evaluate_als(test, recommend_als)
for metric, value in als_metrics.items():
    print(f"{metric}: {value:.4f}")


ALS Metrics:


100%|██████████████████████████████████████| 1000/1000 [00:01<00:00, 638.92it/s]

precision@10: 0.0008
recall@10: 0.0002
hit_rate@10: 0.0030
coverage: 0.0219


### Two Tower

In [133]:
pip install faiss-gpu

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.9 MB/s eta 0:00:000m eta 0:00:010:01:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [137]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [138]:
# 1. Подготовка данных --------------------------------------------------------
# Проверка и очистка данных
print("Проверка данных перед обработкой:")
print(f"Количество строк: {len(df)}, NaN в данных: {df.isnull().sum().sum()}")

# Заполнение пропусков в days_since_prior_order
df['days_since_prior_order'].fillna(0, inplace=True)

Проверка данных перед обработкой:
Количество строк: 1384617, NaN в данных: 0


In [139]:
# Кодирование категориальных признаков
user_features = df.groupby('user_id').agg({
    'order_dow': ['mean', 'std'],
    'order_hour_of_day': ['mean', 'std'],
    'days_since_prior_order': ['mean', 'std'],
    'department': lambda x: x.mode()[0]
}).reset_index()
user_features.columns = ['user_id', 'dow_mean', 'dow_std', 
                        'hour_mean', 'hour_std', 
                        'days_mean', 'days_std', 'fav_dept']

item_features = df.groupby('product_id').agg({
    'aisle': 'first',
    'department': 'first',
    'reordered': 'mean'
}).reset_index()

# Кодируем категории
dept_encoder = LabelEncoder()
aisle_encoder = LabelEncoder()

user_features['fav_dept'] = dept_encoder.fit_transform(user_features['fav_dept'])
item_features['aisle'] = aisle_encoder.fit_transform(item_features['aisle'])
item_features['department'] = dept_encoder.transform(item_features['department'])

# Нормализация числовых признаков
scaler = StandardScaler()
user_features[['dow_mean', 'dow_std', 'hour_mean', 'hour_std', 
              'days_mean', 'days_std']] = scaler.fit_transform(
                  user_features[['dow_mean', 'dow_std', 'hour_mean', 'hour_std',
                               'days_mean', 'days_std']]
              )

In [124]:
# 2. Создание датасета --------------------------------------------------------
class SafeRecommendationDataset(Dataset):
    def __init__(self, interactions, user_features, item_features):
        self.interactions = interactions
        self.user_features = user_features.set_index('user_id')
        self.item_features = item_features.set_index('product_id')
        self.valid_pairs = self._filter_valid_pairs()

    def _filter_valid_pairs(self):
        valid = []
        for user_id, item_id, label in self.interactions:
            if user_id in self.user_features.index and item_id in self.item_features.index:
                valid.append((user_id, item_id, label))
        return valid

    def __len__(self):
        return len(self.valid_pairs)

    def __getitem__(self, idx):
        user_id, item_id, label = self.valid_pairs[idx]
        user_feat = self.user_features.loc[user_id].values
        item_feat = self.item_features.loc[item_id].values
        return {
            'user': torch.tensor(user_feat, dtype=torch.float),
            'item': torch.tensor(item_feat, dtype=torch.float),
            'label': torch.tensor(label, dtype=torch.float)
        }

In [134]:
# 3. Архитектура модели -------------------------------------------------------
class TwoTower(nn.Module):
    def __init__(self, user_dim, item_dim, embedding_dim=64):
        super().__init__()
        
        # User Tower
        self.user_net = nn.Sequential(
            nn.Linear(user_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, embedding_dim)
        )
        
        # Item Tower
        self.item_net = nn.Sequential(
            nn.Linear(item_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, embedding_dim)
        )
        
        # Инициализация весов
        for layer in [self.user_net, self.item_net]:
            for module in layer:
                if isinstance(module, nn.Linear):
                    nn.init.xavier_normal_(module.weight)
                    nn.init.constant_(module.bias, 0.01)

    def forward(self, user, item):
        user_embed = F.normalize(self.user_net(user), p=2, dim=1)
        item_embed = F.normalize(self.item_net(item), p=2, dim=1)
        return torch.sum(user_embed * item_embed, dim=1)

In [135]:
# 4. Обучение модели ----------------------------------------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TwoTower(user_dim, item_dim).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
criterion = nn.BCEWithLogitsLoss()

# Функция для проверки NaN
def check_nan(tensor, name):
    if torch.isnan(tensor).any():
        raise ValueError(f"NaN detected in {name}")

# Цикл обучения
for epoch in range(10):
    model.train()
    total_loss = 0
    
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        user = batch['user'].to(device)
        item = batch['item'].to(device)
        label = batch['label'].to(device)
        
        # Проверка входных данных
        check_nan(user, "User features")
        check_nan(item, "Item features")
        check_nan(label, "Labels")
        
        optimizer.zero_grad()
        outputs = model(user, item)
        
        # Проверка выходов модели
        check_nan(outputs, "Model outputs")
        
        loss = criterion(outputs, label)
        check_nan(loss, "Loss")
        
        loss.backward()
        # Клиппинг градиентов
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f}")

Epoch 1:   0%|                                         | 0/1082 [00:00<?, ?it/s]


ValueError: NaN detected in User features

In [128]:
# 5. Генерация рекомендаций ---------------------------------------------------
def recommend_two_tower(user_id, k=10):
    model.eval()
    
    # Кэшируем все item эмбеддинги
    item_tensors = torch.tensor(item_features.iloc[:, 1:].values, dtype=torch.float).to(device)
    with torch.no_grad():
        item_embeds = model.item_net(item_tensors)
    
    # Эмбеддинг пользователя
    try:
        user_feat = user_features.loc[user_id].values[1:]
    except KeyError:
        return popularity_df.head(k).index.tolist()
    
    user_tensor = torch.tensor(user_feat, dtype=torch.float).to(device).unsqueeze(0)
    with torch.no_grad():
        user_embed = model.user_net(user_tensor)
    
    # Быстрый поиск через FAISS
    import faiss
    index = faiss.IndexFlatIP(64)
    index.add(item_embeds.cpu().numpy())
    distances, indices = index.search(user_embed.cpu().numpy(), k)
    
    return item_features.iloc[indices[0]]['product_id'].tolist()

In [132]:
# 6. Оценка качества ----------------------------------------------------------
# Используем функцию evaluate_item_item из предыдущего кода
print("\nTwo Tower Metrics:")
two_tower_metrics = evaluate_item_item(test, recommend_two_tower)
for metric, value in two_tower_metrics.items():
    print(f"{metric}: {value:.4f}")


Two Tower Metrics:


100%|███████████████████████████████████████| 1000/1000 [00:20<00:00, 48.20it/s]

precision@10: 0.0000
recall@10: 0.0000
hit_rate@10: 0.0000
coverage: 0.0000
